In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D
from IPython.display import display  # to display variables in a "nice" way

# pd.options.display.max_rows = 9999
pd.options.display.max_columns = 200

Setting the Random State:

In [2]:
IDs =  [312920,312919,313385]
rs = np.min(IDs)
np.random.seed(rs)

Exercise 1 (Loading and Preparing the Data):

(a)

In [3]:
# PATH TO THE cla4lsp22_bikez_curated.csv FILE
bikez_path = 'cla4lsp22_bikez_curated.csv'

# LOADING THE DATASET AS DATAFRAME then store in the variable df_tot
df_tot = pd.read_csv(bikez_path)

# DISPLAY OF THE DATAFRAME
# display(df_tot)

##### b) Generate x as a random number: 0, 1, or 2. workdf is the dftot containing only data corresponding to years with reminder r resulted by modulus 3

In [4]:
x = int(np.random.uniform(0,3))
workdf = df_tot[df_tot['Year'] % 3 == x]
display(workdf)

,Brand,Model,Year,Category,Rating,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke,Gearbox,Bore (mm),Stroke (mm),Fuel capacity (lts),Fuel control,Cooling system,Transmission type,Dry weight (kg),Wheelbase (mm),Seat height (mm),Fuel system,Front brakes,Rear brakes,Front tire,Rear tire,Front suspension,Rear suspension
3,acabion,gtbo 600 daytona-vi,2021,Prototype / concept model,NaN,NaN,536.0,NaN,Electric,Electric,Not Given/Unknown,NaN,NaN,NaN,Not Given/Unknown,Liquid,Not Given/Unknown,420.0,NaN,NaN,not given/unknown,single disc,single disc,other,other,not given/unknown,not given/unknown
5,access,37 offroad e-scooter,2021,Scooter,NaN,NaN,1.1,NaN,Electric,Electric,Automatic,NaN,NaN,NaN,Not Given/Unknown,Not Given/Unknown,Chain,42.0,NaN,750.0,not given/unknown,single disc,single disc,other,other,not given/unknown,not given/unknown
14,access,amx 8.57 efi,2018,ATV,NaN,781.0,57.7,NaN,Single cylinder,four-stroke,Automatic,NaN,NaN,21.0,Overhead Cams (OHC),Liquid,Chain,344.0,NaN,NaN,injection,single disc,single disc,25/8-12,25/10-12,other,other
15,access,minisport 110,2021,ATV,NaN,107.0,6.8,NaN,Single cylinder,four-stroke,Automatic,NaN,NaN,NaN,Not Given/Unknown,Air,Chain,95.0,NaN,610.0,not given/unknown,single disc,single disc,other,other,not given/unknown,not given/unknown
16,access,shade 200,2021,ATV,NaN,176.0,13.1,NaN,Single cylinder,four-stroke,Automatic,NaN,NaN,NaN,Overhead Cams (OHC),Air,Chain,201.0,NaN,NaN,not given/unknown,single disc,single disc,other,other,other,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38455,zündapp,roller super,1964,Scooter,NaN,49.0,NaN,NaN,Single cylinder,two-stroke,Not Given/Unknown,39.0,42.0,NaN,Not Given/Unknown,Air,Not Given/Unknown,83.5,NaN,NaN,carburettor. other,expanding brake (drum brake),expanding brake (drum brake),other,other,not given/unknown,not given/unknown
38458,zündapp,roller super,1967,Scooter,NaN,49.0,NaN,NaN,Single cylinder,two-stroke,Not Given/Unknown,39.0,42.0,NaN,Not Given/Unknown,Air,Not Given/Unknown,83.5,NaN,NaN,carburettor. other,expanding brake (drum brake),expanding brake (drum brake),other,other,not given/unknown,not given/unknown
38461,zündapp,z 2 g,1922,Sport,1.5,211.0,2.8,NaN,Single cylinder,two-stroke,2-speed,62.0,70.0,6.0,Not Given/Unknown,Air,Belt,73.0,NaN,NaN,carburettor,not given/unknown,not given/unknown,other,other,other,other
38465,zündapp,z 22,1922,Sport,NaN,211.0,2.3,NaN,Single cylinder,two-stroke,2-speed,62.0,70.0,6.0,Not Given/Unknown,Air,Belt,58.0,NaN,NaN,carburettor,not given/unknown,not given/unknown,other,other,other,other


##### c) Remove randomly from workdf two columns among the features: Front/Rear breaks, Front/Rear tire, Front/Rear suspension.

Denote:
labels: the columns Brand, Model, Year, Category, Rating;
features: all the other ones

In [5]:
labels = workdf.columns[:5].tolist()
features = workdf.columns[5:].tolist()


In [6]:
temp_features = ['Front brakes', 'Rear brakes','Front tire', 'Rear tire','Front suspension', 'Rear suspension']

feat1,feat2 = np.random.choice(temp_features, 2, replace=False)
workdf = workdf.drop(columns=[feat1,feat2])

d) Clean the dataset workdf from missing values in the feature columns (if needed).

##### Show percentage of missing values for each column of workdf

In [7]:
# compute the percentage of NaN values for each column
na_percentage = workdf.isna().mean() * 100
display(na_percentage)

# filter the na_percentage series to only include columns with a percentage of NaN values > 0
na_percentage_filtered = na_percentage[na_percentage > 0]

# subtract the labels list from the list of columns with NaN values > 0
na_features = na_percentage_filtered.index.difference(labels).tolist()

Brand                   0.000000
Model                   0.079789
Year                    0.000000
Category                0.000000
Rating                 42.160696
Displacement (ccm)      2.465491
Power (hp)             31.173701
Torque (Nm)            54.655709
Engine cylinder         0.000000
Engine stroke           0.000000
Gearbox                 0.000000
Bore (mm)              25.412910
Stroke (mm)            25.420889
Fuel capacity (lts)    16.356818
Fuel control            0.000000
Cooling system          0.000000
Transmission type       0.000000
Dry weight (kg)        41.322908
Wheelbase (mm)         33.351951
Seat height (mm)       35.721695
Fuel system             0.000000
Front brakes            0.000000
Rear brakes             0.000000
Rear tire               0.000000
Rear suspension         0.000000
dtype: float64

Remove Nan values from 'Displacement (ccm)' since NaN values are sparse

In [8]:
workdf.dropna(subset=['Displacement (ccm)'], inplace=True)


Fill Nan values from other columns by mean

In [9]:
for col in na_features:
    workdf[col] = workdf[col].fillna(workdf[col].mean())

In [10]:
string_cols = list(workdf.select_dtypes(include=['object']).columns)
df_one_hot_encoded = pd.get_dummies(workdf, columns=string_cols)